In [1]:
from pyspark.sql import SparkSession

# Inicializa uma sessão Spark
spark = SparkSession.builder.appName("CidadesNorte").getOrCreate()



In [2]:
# Lista de cidades do Norte do Brasil e o número de habitantes
dados = [
    ("Manaus", 2219580),
    ("Belém", 1492745),
    ("Porto Velho", 529544),
    ("Boa Vista", 419652),
    ("Palmas", 306296),
    ("Rio Branco", 413418),
    ("Macapá", 503327),
    ("Santarém", 294447),
]

# Define o schema para o DataFrame
schema = ["Cidade", "Habitantes"]

# Cria o DataFrame
df = spark.createDataFrame(dados, schema)

# Mostra o DataFrame
df.show()

+-----------+----------+
|     Cidade|Habitantes|
+-----------+----------+
|     Manaus|   2219580|
|      Belém|   1492745|
|Porto Velho|    529544|
|  Boa Vista|    419652|
|     Palmas|    306296|
| Rio Branco|    413418|
|     Macapá|    503327|
|   Santarém|    294447|
+-----------+----------+



In [3]:
#Criando nossa primeira tabela delta
df.write.format("delta").save("s3a://lakehouse-demo/cidades_norte")

In [4]:
#Optimizando a tabela
from delta import DeltaTable

delta_table = DeltaTable.forPath(spark, "s3a://lakehouse-demo/cidades_norte")
delta_table.optimize().executeCompaction()

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [13]:
#Versionamento
delta_table.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      1|2023-09-16 12:48:51|  null|    null| OPTIMIZE|{predicate -> [],...|null|    null|     null|          0|SnapshotIsolation|        false|{numRemovedFiles ...|        null|Apache-Spark/3.3....|
|      0|2023-09-16 12:40:27|  null|    null|    WRITE|{mode -> ErrorIfE...|null|    null|     null|       null|     Serializable|         true|{numFiles -> 9, n...|        null|Apache-Spark/3.3....|


In [9]:
#Vacuum

delta_table.vacuum(0)

IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
       

In [10]:
spark = SparkSession.builder.appName("CidadesNorte")\
                             .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")\
                             .getOrCreate()

In [11]:

#Vacuum
delta_table.vacuum(0)

DataFrame[]